## A very brief introduction to image processing and particle image velocimetry (PIV)

First, we begin with importing a number of libraries. These include:
* [numpy](https://numpy.org/)
* [skimage](https://scikit-image.org/)
* [matplotlib](https://matplotlib.org/)
* [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)
* [openpiv](http://www.openpiv.net/)

<br>
If you get an error message in regards to the openpiv or numdifftools then you will need to do the following:<br>
open up an [Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/#cli-hello). Then just type: <br>
> `pip install numdifftools`
<br>

<br>You'll also need to install the Python package [openpiv](https://github.com/OpenPIV/openpiv-python). To install, you'll again open up an Anaconda Prompt and type: <br>
> `conda install -c conda-forge openpiv`

In [9]:
from io import BytesIO
import numpy as np
import pandas as pd
from ipywidgets import Layout, interact, fixed, interactive
from IPython.display import Image
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import sys
from scipy import ndimage
from scipy.ndimage import gaussian_filter1d as gf1d
from skimage import measure
%matplotlib notebook
import matplotlib
from matplotlib import pylab
import skimage
from skimage import filters, feature, io
from matplotlib import cm
from matplotlib.pyplot import figure
from skimage.filters import roberts, sobel, sobel_h, sobel_v, scharr, \
    scharr_h, scharr_v, prewitt, prewitt_v, prewitt_h
import numdifftools as nd
from openpiv import tools, scaling, pyprocess, validation, filters
from openpiv import piv

ImportError: cannot import name simple_piv

The following is just a function to help us display images. Don't worry about going through the code within.

In [2]:
#This is just a function to help us go from an array to another type of
#  data structure that can be more easily displayed

def arr2img(arr):
    """Display a 2- or 3-d numpy array as an image."""
    if arr.ndim == 2:
        format, cmap = 'png', matplotlib.cm.gray
    elif arr.ndim == 3:
        format, cmap = 'jpg', None
    else:
        raise ValueError("Only 2- or 3-d arrays can be displayed as images.")
    # Don't let matplotlib autoscale the color range so we can control overall luminosity
    if arr.dtype == 'uint8':
        vmax=255
        vmin=0
    elif arr.dtype == 'uint16':
        vmin = arr.min()
        vmax = arr.max()
    else:
        vmax=arr.max()
        vmin=arr.min()
    with BytesIO() as buffer:
        matplotlib.image.imsave(buffer, arr, format=format, cmap=cmap, vmin=vmin, vmax=vmax)
        out = buffer.getvalue()
    return Image(out)

We will start by analyzing the [image stack found here](https://drive.google.com/file/d/1AmaQB89vuirshH-MIT0jL_hSv6DMj0_g/view?usp=sharing). This is the same movie used in the Fiji/ImageJ tutorial.<br>
If you haven't yet, download that tif file and save it somewhere to your computer. <br>
In the block of code below, specify where to find the tif file (replace the current path saved as `data_directory`).

In [3]:
data_directory = 'C:\\Users\\Ryan\\Downloads\\'
filename = 'actin network contracting over 6 min.tif'

In [4]:
#Read in the image file
im=skimage.io.imread(data_directory+filename)
print("Shape of the image data file: (%i,%i,%i)" % im.shape)
print("So each image is %i-by-%i pixels large and we have %i frames in our movie." % (im.shape[1], im.shape[2], im.shape[0]))

Shape of the image data file: (77,256,256)
So each image is 256-by-256 pixels large and we have 77 frames in our movie.


C:\ProgramData\Anaconda2\lib\site-packages\skimage\external\tifffile\tifffile.py:2128: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")


If a `warning` message popped up when you ran the above block of code, don't worry. Only `error` messages mean that the code did not run. 

In [27]:
def edit_image_using_filter(image_stack, im_number, sigma=1, gamma=1):
    '''
    This function takes an image stack and returns a filtered image. 
    We apply a Gaussian image filter. 
    Scaling of the image is also done with a gamma factor. By reducing gamma below 
    1, low intensity regions of the image stand out more.
    '''
    
    new_image = ndimage.gaussian_filter(image_stack[im_number], sigma)
    new_image = new_image**gamma
    return arr2img(new_image)

v=interact(edit_image_using_filter, image_stack=fixed(im), im_number=(0,im.shape[0]-1,1), sigma=(0,10,0.5), 
          gamma=(0.01,1,0.01))
display(v)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0zOCwgZGVzY3JpcHRpb249dSdpbV9udW1iZXInLCBtYXg9NzYpLCBGbG9hdFNsaWRlcih2YWx1ZT0xLjAsIGRlc2NyaXB0aW/igKY=


<function __main__.edit_image_using_filter>

In [8]:
# Select frame1 and a step size
frame1 = 1
stepsize=5
frame2 = frame1+stepsize
x,y,u,v = piv.simple_piv(im[frame1], im[frame2])

AttributeError: 'module' object has no attribute 'simple_piv'

In [ ]:
piv.piv.